In [3]:
import numpy as np
import matplotlib.pyplot as plt
from database_methods import Table, to_show, head
import sqlite3
con = sqlite3.connect('database_results.db')
cursor = con.cursor()

In [4]:
mas = Table(connection=con, name='masters')
exp = Table(con, name = 'experiments')
meas = Table(con, name = 'measurements')

In [19]:
print(mas.columns)
print(exp.columns)
print(meas.columns)
print('-'*80)
mas.show_all()
exp.show_all()

CATEGORY, PERIOD, AMPLITUDE, MASTER, FWHM, R, S
DATE, MASTER_ID, DIP, FOLDER
EXPERIMENT_ID, NP, WAVELENGTH, FILE, AR_TH, AR_DIP, AR_NOISE, EXC_THS, RATE, INFO, COUNT, X, Y, CON, INP, INPPX, IBGPX, STD, stdX, stdY, stdCON, stdINP, stdINPPX, stdIBGPX, stdSTD, NOTES, AR_MIN, AR_COND, EXP_TYPE
--------------------------------------------------------------------------------
ID	CATEGORY	PERIOD	AMPLITUDE	MASTER	FWHM	R	S
1	600	405	45	M5	23.0	8.0	15.9
2	600	405	53	P3	25.0	19.7	3.8
3	650	452	32	K4	12.5	11.9	17.8
4	650	452	40	K5	15.0	18.0	8.8
5	750	528	24	L3	7.0	13.8	78.5
6	750	526	30	Q4	9.5	22.0	30.7
ID	DATE	MASTER_ID	DIP	FOLDER
1	20-02-25	2	615	20_02_25_P3/
2	20-02-26	6	727	20_02_26_Q3/
3	20-03-16	4	658	20_03_16_K5/
5	20-02-25	1	598	20_02_25_M5/
6	20-03-16	3	647	20_03_16_K4/
7	20-04-15	5	740	20_04_15_L3/
8	20-04-14	1	606	20_04_14_M5/


In [18]:
con.commit()

In [34]:
# 
columns = 'MASTER,  AVG(DIP), AVG(INP), AVG(INP/IBGPX), AVG(CON), AVG(INPPX), AVG(IBGPX)'
data = con.execute("""
SELECT {}
FROM 'masters' as MAS 
INNER JOIN 'experiments' as EXP 
ON MAS.ID = EXP.MASTER_ID
INNER JOIN 'measurements' as MEAS
ON EXP.ID = MEAS.EXPERIMENT_ID
GROUP BY MASTER
ORDER BY DIP DESC
;
   """.format(columns))

head(columns)

for row in data:
    print(str(to_show(row)).replace(', ', '\t').replace("'", "")[1:-1])


MASTER	 AVG(DI	AVG(INP	AVG(INP	AVG(CON	AVG(INP	AVG(IBG
L3	740.00	18.53	54.94	3.31	1.15	0.3505
Q4	727.00	25.81	58.02	3.81	1.73	0.4763
K5	658.00	14.79	48.99	5.44	1.64	0.3078
K4	647.00	6.18	24.67	3.32	0.9056	0.2691
P3	615.00	5.25	20.50	7.27	1.81	0.2621
M5	606.00	2.54	11.68	6.60	1.42	0.2196


In [52]:
# 
columns = ' MASTER, DATE, FILE, WAVELENGTH, NP, DIP, COUNT, RATE, CON, COUNT, X*Y, INP, INPPX, IBGPX, STD, EXP_TYPE,  NOTES'
data = con.execute("""
SELECT {}
FROM 'masters' as MAS 
INNER JOIN 'experiments' as EXP 
ON MAS.ID = EXP.MASTER_ID
INNER JOIN 'measurements' as MEAS
ON EXP.ID = MEAS.EXPERIMENT_ID
WHERE category = 600

ORDER BY CON DESC
;
   """.format(columns))

head(columns)

i=0
for row in data:
    print(str(to_show(row)).replace(', ', '\t').replace("'", "")[1:-1])
    i+=1
    
    
print('---\n{}'.format(i))

 MASTER	DATE	FILE	WAVELEN	NP	DIP	COUNT	RATE	CON	COUNT	X*Y	INP	INPPX	IBGPX	STD	EXP_TYP	 NOTES
P3	20-02-2	raw_08_	615	80	615	51	25.33	9.08	51	27.17	4.61	1.87	0.2231	0.2916	-	-
P3	20-02-2	raw_16_	612	60	615	88	40.74	8.28	88	26.76	4.66	1.80	0.2225	0.2879	-	nice	F
P3	20-02-2	raw_29_	612	80	615	47	30.00	8.16	47	32.51	4.91	1.63	0.2032	0.2629	sh	nice
P3	20-02-2	raw_09_	614	80	615	51	43.18	8.03	51	28.83	4.36	1.67	0.2244	0.2938	-	-
P3	20-02-2	raw_10_	612	80	615	99	46.43	7.86	99	30.26	4.53	1.50	0.1912	0.2470	-	nice
P3	20-02-2	raw_06_	622	80	615	47	20.00	7.80	47	28.36	5.34	2.00	0.2645	0.3476	-	-
M5	20-04-1	raw_11_	616	80	606	69	19.79	7.56	69	11.39	1.76	1.37	0.1836	0.2328	-	nice
M5	20-04-1	raw_10_	618	80	606	47	13.27	7.54	47	16.51	3.39	1.51	0.2018	0.2733	-	-
P3	20-02-2	raw_32_	612	80	615	348	168.45	7.43	348	31.24	4.54	1.49	0.2021	0.2650	strep	nice
P3	20-02-2	raw_03_	628	80	615	39	25.00	6.86	39	32.14	6.73	2.22	0.3297	0.4307	-	-
P3	20-02-2	raw_07_	618	80	615	49	26.70	6.82	49	43.26	5.85	1.71	0.2803	0.

In [57]:
columns = 'NP, WAVELENGTH, AR_TH, AR_NOISE, EXC_THS, AR_MIN, AR_COND'
data = con.execute("""
SELECT {} FROM 'measurements'
WHERE WAVELENGTH=732
ORDER BY WAVELENGTH ASC;
   """.format(columns))

print(columns.replace(', ','\t'))
for row in data:
    print(str(row).replace(', ', '\t').replace("'", "")[1:-1])

NP	WAVELENGTH	AR_TH	AR_NOISE	EXC_THS	AR_MIN	AR_COND
80	732	60.0	0.001	[2]	10	2
60	732	59.0	0.001	[2]	10	2
60	732	46.0	0.001	[2]	10	2
40	732	12.0	0.001	[1.1]	2	2
80	732	100.0	0.001	[2]	2	2
60	732	20.0	0.001	[2]	2	2
80	732	40.0	0.001	[2]	2	2


In [213]:
con.execute("""
UPDATE 'measurements' SET NOTES = 'weak' WHERE COUNT = 165
""")

In [215]:
meas.commit()

In [9]:
con.close()